In [0]:
%pip install kaggle
   

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.7/12.7 MB 120.3 MB/s eta 0:00:00
Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [0]:
import os

os.environ["KAGGLE_USERNAME"] = "your_kaggle_username"
os.environ["KAGGLE_KEY"] = "your_kaggle_api_key"

print("Kaggle credentials configured!")

Kaggle credentials configured!


In [0]:
import os

os.environ["KAGGLE_USERNAME"] = "Pornimaa"
os.environ["KAGGLE_KEY"] = "e647171cc55b66cdabe9eb5930b38f77"

print("Kaggle credentials configured!")

Kaggle credentials configured!


3. Create Database Schema


In [0]:

spark.sql("""
CREATE SCHEMA IF NOT EXISTS workspace.ecommerce
""")

DataFrame[]

4.Create Volume for Data Storage

In [0]:
spark.sql("""
CREATE VOLUME IF NOT EXISTS workspace.ecommerce.ecommerce_data
""")

DataFrame[]

5.Download Dataset from Kaggle(to Volume)

In [0]:
%sh
cd /Volumes/workspace/ecommerce/ecommerce_data
kaggle datasets download -d mkechinov/ecommerce-behavior-data-from-multi-category-store

Dataset URL: https://www.kaggle.com/datasets/mkechinov/ecommerce-behavior-data-from-multi-category-store
License(s): copyright-authors
ecommerce-behavior-data-from-multi-category-store.zip: Skipping, found more recently modified local copy (use --force to force download)


6. Extract Downloaded Dataset

In [0]:
%sh
cd /Volumes/workspace/ecommerce/ecommerce_data
unzip -o ecommerce-behavior-data-from-multi-category-store.zip
ls -lh

Archive:  ecommerce-behavior-data-from-multi-category-store.zip
  inflating: 2019-Nov.csv            
  inflating: 2019-Oct.csv            
total 18G
-rwxrwxrwx 1 spark-62f33b71-b326-4a1e-9d29-cc nogroup 8.4G Jan  9 00:38 2019-Nov.csv
-rwxrwxrwx 1 spark-62f33b71-b326-4a1e-9d29-cc nogroup 5.3G Jan  9 00:40 2019-Oct.csv
-rwxrwxrwx 1 spark-62f33b71-b326-4a1e-9d29-cc nogroup 4.3G Jan  9 00:36 ecommerce-behavior-data-from-multi-category-store.zip


 Clean Up Zip File

In [0]:
%sh
cd /Volumes/workspace/ecommerce/ecommerce_data
rm -f ecommerce-behavior-data-from-multi-category-store.zip
ls -lh

total 14G
-rwxrwxrwx 1 nobody nogroup 8.4G Jan  9 00:38 2019-Nov.csv
-rwxrwxrwx 1 nobody nogroup 5.3G Jan  9 00:40 2019-Oct.csv


8. Restart Python Environment

In [0]:
%restart_python

9. Load November 2019 Data

In [0]:
df_n = spark.read.csv("/Volumes/workspace/ecommerce/ecommerce_data/2019-Nov.csv")

10. Load October 2019 Data

In [0]:
df = spark.read.csv("/Volumes/workspace/ecommerce/ecommerce_data/2019-Oct.csv")

11. Display Dataset Statistics and Schema

In [0]:

print(f"October 2019 - Total Events: {df.count():,}")
print("\n" + "="*60)
print("SCHEMA:")
print("="*60)
df.printSchema()


October 2019 - Total Events: 42,448,765

SCHEMA:
root
 |-- _c0: string (nullable = true)
 |-- _c1: string (nullable = true)
 |-- _c2: string (nullable = true)
 |-- _c3: string (nullable = true)
 |-- _c4: string (nullable = true)
 |-- _c5: string (nullable = true)
 |-- _c6: string (nullable = true)
 |-- _c7: string (nullable = true)
 |-- _c8: string (nullable = true)



12. Display Sample Data

In [0]:
nov_events = spark.read.csv(
    "/Volumes/workspace/ecommerce/ecommerce_data/2019-Nov.csv",
    header=True,
    inferSchema=True

)
oct_events = spark.read.csv(
    "/Volumes/workspace/ecommerce/ecommerce_data/2019-oct.csv",
    header=True,
    inferSchema=True
)

Expected Schema

In [0]:
display(nov_events.schema)

StructType([StructField('event_time', TimestampType(), True), StructField('event_type', StringType(), True), StructField('product_id', IntegerType(), True), StructField('category_id', LongType(), True), StructField('category_code', StringType(), True), StructField('brand', StringType(), True), StructField('price', DoubleType(), True), StructField('user_id', IntegerType(), True), StructField('user_session', StringType(), True)])

In [0]:
display(nov_events.limit(10))

event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
2019-11-01T00:00:00.000Z,view,1003461,2053013555631882655,electronics.smartphone,xiaomi,489.07,520088904,4d3b30da-a5e4-49df-b1a8-ba5943f1dd33
2019-11-01T00:00:00.000Z,view,5000088,2053013566100866035,appliances.sewing_machine,janome,293.65,530496790,8e5f4f83-366c-4f70-860e-ca7417414283
2019-11-01T00:00:01.000Z,view,17302664,2053013553853497655,null,creed,28.31,561587266,755422e7-9040-477b-9bd2-6a6e8fd97387
2019-11-01T00:00:01.000Z,view,3601530,2053013563810775923,appliances.kitchen.washer,lg,712.87,518085591,3bfb58cd-7892-48cc-8020-2f17e6de6e7f
2019-11-01T00:00:01.000Z,view,1004775,2053013555631882655,electronics.smartphone,xiaomi,183.27,558856683,313628f1-68b8-460d-84f6-cec7a8796ef2
2019-11-01T00:00:01.000Z,view,1306894,2053013558920217191,computers.notebook,hp,360.09,520772685,816a59f3-f5ae-4ccd-9b23-82aa8c23d33c
2019-11-01T00:00:01.000Z,view,1306421,2053013558920217191,computers.notebook,hp,514.56,514028527,df8184cc-3694-4549-8c8c-6b5171877376
2019-11-01T00:00:02.000Z,view,15900065,2053013558190408249,null,rondell,30.86,518574284,5e6ef132-4d7c-4730-8c7f-85aa4082588f
2019-11-01T00:00:02.000Z,view,12708937,2053013553559896355,null,michelin,72.72,532364121,0a899268-31eb-46de-898d-09b2da950b24
2019-11-01T00:00:02.000Z,view,1004258,2053013555631882655,electronics.smartphone,apple,732.07,532647354,d2d3d2c6-631d-489e-9fb5-06f340b85be0


In [0]:
nov_events = spark.read.csv(
    "/Volumes/workspace/ecommerce/ecommerce_data/2019-Nov.csv",
    header=True,
    inferSchema=True

)
oct_events = spark.read.csv(
    "/Volumes/workspace/ecommerce/ecommerce_data/2019-oct.csv",
    header=True,
    inferSchema=True
)

In [0]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window

In [0]:
revenue = oct_events.filter(F.col("event_type") == "purchase") \
    .groupBy("product_id", "product_name") \
    .agg(F.sum("price").alias("revenue")) \
    .orderBy(F.desc("revenue")).limit(5)

---------------------------------------------------------------------------
NameError                                 Traceback (most recent call last)
File <command-4699002970824100>, line 1
----> 1 revenue = oct_events.filter(F.col("event_type") == "purchase") \
      2     .groupBy("product_id", "product_name") \
      3     .agg(F.sum("price").alias("revenue")) \
      4     .orderBy(F.desc("revenue")).limit(5)

NameError: name 'oct_events' is not defined